# Redes de Computadores

## Trabalho Prático Tp1
### MpegDashProxy
draft: 0.1

O objetivo deste trabalho é desenvolver um *proxy* HTTP para suporte à reprodução num browser de filmes disponibilizados segundo os princípios do [MPEG-DASH](https://en.wikipedia.org/wiki/Dynamic_Adaptive_Streaming_over_HTTP) (Dynamic Adaptable Streaming over HTTP).

### Contexto
Um servidor HTTP disponibiliza filmes, cada um codificado em diversas qualidades ou resoluções (bitrates). Cada versão de um filme é denominada de pista (*track*) e é composta por segmentos com duração **idêntica** de alguns segundos, mas de dimensão diferente de acordo com a *bitrate* utilizada. A codificação das pistas foi realizada de modo a que o número total de segmentos e a sua duração seja igual em todas as pistas de um mesmo filme. Adicionalmente, temporalmente, os segmentos estão perfeitamente alinhados entre pistas, pelo que são funcionalmente equivalentes.

A reprodução de um filme realiza-se no *browser* por recurso a um reprodutor (*player*) *javascript*. Para que a reprodução de um filme seja correta, é necessário que os segmentos de um filme sejam entregues ao *player* **atempadamente**, na ordem correta, mas independentemente da pista a que pertencem. 

O diálogo entre o *browser* (*player* *javascript*) e o servidor HTTP onde estão armazenados os filmes será mediado por um *proxy* HTTP. A função deste *proxy* será adequar a qualidade dos segmentos entregues ao *player*, dinamicamente, em função do débito da rede disponível, mas evitando que a reprodução sofra interrupções por falta de dados. Monitorizando o estado da rede, em momentos favoráveis, espera-se que o *proxy* dê prioridade aos segmentos
de pistas de qualidade mais elevada, fazendo o oposto para reagir à degradação do débito da rede ou havendo o risco de o *player* estar prestes a parar. Neste último caso, optando por transferir do servidor segmentos de pior qualidade, mas de menor dimensão.

O diagrama, abaixo, ilustra um esboço do protocolo de comunicação entre os componentes, mostrando que depois de obter a descrição de um filme, contida num ficheiro *manifest.txt*, o *proxy* responde
aos sucessivos pedidos do *browser* com segmentos em sequência crescente, mas de pistas diferentes.

<img src="https://docs.google.com/drawings/d/e/2PACX-1vSlfmLzJb9VxMaue_znHJArdw0q78iwoy6tWM2xu2KS95ty-QgVAhyOTAShsblx0jPqZo-9m5VMUP0D/pub?w=1438&h=704" width="75%"></img>



### Materiais fornecidos

É fornecido [código Java 17](https://github.com/smduarte/RC2021-labs/tree/main/tp1/RC2021-MpegDash-tp1) com um esqueleto da solução. A faceta servidor do *proxy* HTTP, que implementa a interação entre o *browser* e o *proxy* está completa, ficando a faltar a faceta cliente HTTP que dialoga com o servidor que aloja os filmes e implementa a heurística de escolha dos segmentos a transferir.

A proposta de solução está concebida segundo um modelo *produtor/consumidor* baseado na partilha de uma fila bloqueante. Neste quadro, o *consumidor* corresponde à faceta servidor do *proxy* que atende os pedidos do *browser*, consumindo da fila os segmentos do filme que entrega ao *browser*. No *proxy*, o *produtor* corresponde ao cliente HTTP do servidor que aloja os filmes e que obtém dados de segmentos e os coloca na fila logo que estão disponíveis. 

A cada nova reprodução de um filme, no código fornecido, o consumidor já é lançado automaticamente num *thread* dedicado. Consequentemente, uma vez obtida e interpretada a descrição do filme, o *produtor* pode antecipar-se e pedir segmentos ao servidor de filmes ao ritmo que entender. 

O código *produtor* é a parte a completar neste trabalho e centra-se na lógica para descarregar os segmentos na melhor qualidade admissível face às condições da rede no momento, usando pedidos HTTP parciais.

Adicionalmente são fornecidos:

+ Uma página *web* que implementa o *player* e que será o cliente do *proxy* HTTP a completar;
+ Um servidor HTTP para alojar os filmes e a página web do *player*;
+ Dois pequenos filmes codificados em várias qualidades, nos formatos adequados;
+ Imagens *Docker* para executar o servidor HTTP, simulando condições de rede variáveis.

### Especificações adicionais

No servidor, cada filme está alojado numa pasta dedicada com o mesmo nome do filme. A pasta de um filme
contém o ficheiro **manifest.txt** que descreve as pistas disponíveis, incluindo os limites (*offsets*) dos
segmentos dentro de cada pista, por ordem crescente de qualidade (*bitrate*). Cada pista corresponde a um ficheiro separado, com todos os segmentos agrupados e indexáveis pelos limites indicados no *manifest*. Transferir um segmento de uma pista consiste em fazer um pedido HTTP parcial do ficheiro associado a uma pista com o intervalo adequado.   

O formato do ficheiro **manifest.txt** pode ser inferido pelo código fornecido que faz o seu processamento, contido na classe [MovieManifest](https://github.com/smduarte/RC2021-labs/blob/main/tp1/RC2021-MpegDash-tp1/src/media/MovieManifest.java). Esta classe é executável e contém um exemplo.

O servidor HTTP que aloja os filmes e a página do *player javascript* é executado por via de uma imagem docker, utilizando um dos seguintes comandos:

`docker run -ti -p 9999:8080 smduarte/rc2021-tp1-test1`

`docker run -ti -p 9999:8080 smduarte/rc2021-tp1-test2`

No primeiro caso, o servidor é lançado num cenário de rede em que o débido é constante e suficiente para reproduzir os filmes na melhor qualidade disponível. No segundo caso, o débito da rede varia ao longo do tempo e tem como objetivo testar a capacidade de adaptação do algoritmo desenvolvido.

Uma vez lançado o servidor, o *player* pode ser acedido no seguinte url:

[http://localhost:9999/index.html](http://localhost:9999/index.html)

Os conteúdos alojados pelo servidor podem ser consultados no *browser* abrindo o url [http://localhost:9999/](http://localhost:9999/) Os dois filmes disponibilizados estão guardados nas pastas [coco](http://localhost:9999/coco) e [dante](http://localhost:9999/dante).

Os pedidos HTTP *browser* realizados pelo *player* *javascript* são dirigidos a http://localhost:1234, pelo que o **porto 1234** utilizado pelo *proxy* **não** pode ser alterado.

Nota: **O *proxy* não pode utilizar pedidos HTTP em paralelo para aumentar o débito da rede.**

### Critérios de avaliação

A solução apresentada será avaliada segundo os seguintes objetivos:

1. **Solução não adaptativa**: [Valorização máxima: 12 valores]

    É possível reproduzir os filmes, sem pausas, obtendo os segmentos sempre da mesma pista.
    <br><br>
2. **Solução adaptativa**: [Valorização máxima: 17 valores]

    É possível reproduzir os filmes, sem pausas, obtendo os segmentos das pistas disponíveis, dinamicamente,
    em função da qualidade da rede.
    <br><br>
    
3. **Versão adaptativa usando Http 1.1**: [Valorização máxima: 20 valores]

    A solução é adaptativa e faz os pedidos ao servidor de filmes utilizando a versão 1.1 do protocolo
    HTTP.
    
A qualidade da programação poderá ser tida em conta na avaliação do trabalho, em particular para a obtenção da valorização máxima.

IMPORTANTE: A solução apresentada deve procurar ser **genérica**, ou seja não ser afinada especificamente para os filmes e condições de rede fornecidos.

### Entrega

O prazo de entrega é dia 13 de Novembro, às 23h59 minutos. Entregas atrasadas poderão ser recusadas e caso venham ser aceites serão alvo de penalização.

As instruções para a entrega do trabalho serão oportunamente adicionadas à versão online do enunciado, disponível [aqui](https://github.com/smduarte/RC2021-labs/blob/main/tp1/MpegDashProxy-tp1.ipynb). 

### Adenda

(a completar)